In [ ]:

import os
import sys
import io
from PIL import Image
import warnings
import pytesseract
import cv2
import easyocr
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from datasets import Dataset
import time
import numpy as np


# zeige keine Warnungen an
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


from src.utils import rotate_image, pil_to_cv, from_cv_to_pil
from src.ocr_pipeline import OCRPreprocessor, OCRPostProcessor

In [ ]:
# Laden des Datensatzes
test_dataset = Dataset.load_from_disk("../data/raw/test")

In [ ]:
example = test_dataset[0]

In [ ]:
%%time
# Anwenden der OCR Vorverarbeitung
preprocessor = OCRPreprocessor(image)
preprocessor.cropping(buffer_size=10)
preprocessor.resize(factor=3)
preprocessor.contrast_stretching()
preprocessor.power_law_transform(gamma=2)
preprocessor.to_gray()
preprocessor.correct_skew()
preprocessor.sharpen(kernel_type="laplace_standard")
preprocessor.opening(kernel=(1,1), iterations=2)
preprocessor.power_law_transform(gamma=2)
preprocessor.resize(factor=0.5)
processed_image_small = preprocessor.get_image()

In [ ]:
%%time
pytesseract_output = pytesseract.image_to_string(processed_image_big)
pytesseract_output

In [ ]:
if pytesseract_output.strip():  # Prüft, ob `ocr_output` nicht leer ist
    postprocessor = OCRPostProcessor(pil_standardytesseract_output)
    # Anwenden verschiedener Methoden
    postprocessor.identify_language()
    postprocessor.remove_special_characters()
    postprocessor.remove_stopwords()
    #postprocessor.stem()
    #postprocessor.lemmatize()
    postprocessor.lowercase()
    postprocessor.spellcheck(checker_type="symspell")
            
    # Aufbereiteten OCR-Output extrahieren
    ocr_output = postprocessor.get_text()
else: ocr_output = "no text found with ocr!"

ocr_output